In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, lit

In [ ]:
spark = SparkSession.builder.appName("bk-imp").getOrCreate()

In [ ]:
df = spark.read.json("../data/meta_Digital_Music.json").select(
    ["also_buy", "also_view", "asin"]
)

In [ ]:
df.printSchema()

In [ ]:
# Explode also_buy
also_buy_df = df.select(
    col("asin").alias("src_product_id"),
    explode("also_buy").alias("dst_product_id"),
    lit("same_buyer").alias("relationship"),
)

# Explode also_view
also_view_df = df.select(
    col("asin").alias("src_product_id"),
    explode("also_view").alias("dst_product_id"),
    lit("same_viewer").alias("relationship"),
)

# Union the two dataframes
result_df = also_buy_df.union(also_view_df).dropDuplicates(
    ["src_product_id", "dst_product_id"]
)

In [ ]:
result_df.show(10)